<a href="https://colab.research.google.com/github/menon92/DL-Sneak-Peek/blob/master/%E0%A6%9F%E0%A7%87%E0%A6%A8%E0%A7%8D%E0%A6%B8%E0%A6%B0%E0%A6%AB%E0%A7%8D%E0%A6%B2%E0%A7%8B_%E0%A7%A8_%E0%A7%A6_%E0%A6%93_%E0%A6%95%E0%A7%87%E0%A6%B0%E0%A6%BE%E0%A6%B8_%E0%A6%AA%E0%A6%B0%E0%A6%BF%E0%A6%9A%E0%A6%BF%E0%A6%A4%E0%A6%BF_%E0%A6%AA%E0%A6%B0%E0%A7%8D%E0%A6%AC_%E0%A7%AA_%5B_%E0%A6%87%E0%A6%AE%E0%A7%87%E0%A6%9C_%E0%A6%A1%E0%A6%BE%E0%A6%9F%E0%A6%BE_%E0%A6%B2%E0%A7%8B%E0%A6%A1_%E0%A6%95%E0%A6%B0%E0%A6%BE_%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ইমেজ ডাটা 

এখানে আমরা দেখব ডিপলার্নিং মডেল ট্রেনিং করার আগে ডাটা লোডিং পাইপলাইন কিভাবে করা হয় । এইজন্য ডাটাসেট হিসাবে ব্যাবহার করব 
<a herf="https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"> flower photos</a>। এটা 218 MB সাইজের একটা ডাটাসেট। এই ডাটাসেট ব্যাবহার করে আমরা নিচের বিষয়গুলো কভার করার চেষ্টা করব । 

* `tf.keras.preprocessing.image.ImageDataGenerator` দিয়ে কিভাবে ইমেজ ডাটা লোড করা যায় । 
* `tf.data` দিয়ে কিভাবে ইমেজ ডাটা লোড করা যায় । 
* উপরের দুইটার কোনটাতে কি সুবিধা পাওয়া যাবে । 
* কোনটাতে কেমন সময় লাবে । 

### প্রয়োজনীয় প্যাকেজ ইম্পোট করে নেই 

In [0]:
try:
  # %tensorflow_version এই কমান্ড কেবলমাত্র colab এ কাজ করে লোকাল নোটবুকে কাজ করবে না । 
  # এই জন্য try except ব্যাবহার করা হয়েছে । 
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

In [0]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [0]:
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

In [0]:
# টেন্সফ্লো ভার্সন চেক করে নিন 
tf.__version__

### ডাটাসেট ডাউনলোড 

ডাটাসেট ডাউনলোড করার জন্য আমরা `tf.keras.utils.get_file` এই ফাংশন ব্যাবহার করব । এখানে আমরা ডাটাসেট url এবং কি নামের (`fname='flower_photos'`) ফোল্ডারে ডাউনলোড হয়ে জমা হবে সেটা বলে দিচ্ছি । এবং `flower_photos.tgz` যেহেতু একটা জিপ ফাইল তাই এটাকে আনজিপ করার জন্য আমরা `untar=True` দিয়ে দিব । 

In [0]:
import pathlib

DATASET_URL = 'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz'

data_dir = tf.keras.utils.get_file(origin=DATASET_URL, fname='flower_photos', untar=True)
data_dir = pathlib.Path(data_dir)
print('Dataset directory:', data_dir)

In [0]:
!sudo  apt -qq install tree
!tree -L 1 /root/.keras/datasets/flower_photos

আমরা দেখতে পাচ্ছি যে ৫ টা আলাদা আলাদা ফুলের ফোল্ডার আছে এবং একটা টেক্সট ফাইল আছে । 

In [0]:
image_count = len(list(data_dir.glob('*/*.jpg')))
image_count

আমরা দেখতে পাচ্ছি যে সব মিলে ৩৬৭০ টা ফুলের ছবি আছে । এখন আমরা দেখব যে কোন ফুল কয়টা করে আছে । 

In [0]:
FLOWERS = ['daisy', 'dandelion', 'roses','sunflowers','tulips']
for flower in FLOWERS:
    total_flower = len(list(data_dir.glob(flower + "/*.jpg")))
    print("`{:10s}` folder contain {} flower images".format(flower, total_flower))

আমরা কিছু ৩ টা টিউলিপ ফুল প্লট করে দেখি । আমরা প্রত্যেকটা ফুলের ছবির সাথে তাদের সাইজ প্রিন্ট করেছি । এবং আমরা দেখতে পাচ্ছি যে এক একটা ছবির সাইজ এক এক রকম । আমাদের মডেলে ডটা ফিট করার আগে অবশ্যই সব ছবিকে একটা ফিক্স সাইজে নিয়ে আসতে হবে । 

In [0]:
roses = list(data_dir.glob('tulips/*'))

for image_path in roses[:3]:
    image = Image.open(str(image_path))
    print('image size', image.size)
    display.display(image)

## `keras.preprocessing` দিয়ে ইমেজ ডাটা লোড করা

আমরা <a herf="https://keras.io/preprocessing/image/"> ImageDataGenerator </a> ব্যাবহার করে খুব সহজেই ইমেজ লোড করতে পারি । `rescale=1./255` দিয়ে আমরা বলে দিচ্ছি ইমেজের পিক্সেল ভেলু গুলোকে [0,255] থেকে [0,1] মাঝে নিয়ে আসা ।  

In [0]:
# The 1./255 is to convert from uint8 to float32 in range [0,1].
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

ইমেজ ডাটা লোড করার জন্য আমরা কিছু প্যারামিটার ডিফাইন করি । 

In [0]:
BATCH_SIZE = 32
IMG_HEIGHT = 224
IMG_WIDTH = 224
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)

এখন আমরা ডাটা ডিরেক্টরি থেকে ৫ টা ক্লাসের নাম বের করে ফেলি ।   

In [0]:
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"])
CLASS_NAMES

In [0]:
train_data_gen = image_generator.flow_from_directory(
    directory=str(data_dir),
    batch_size=BATCH_SIZE,
    color_mode='rgb', # one of "grayscale", "rgb", "rgba". Default: "rgb". 
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    classes = list(CLASS_NAMES),
    class_mode='categorical' # "categorical", "binary", "sparse", "input", or None. Default: "categorical".
)

আমরা যেহেতু ইমেজ ডিরেক্টরি থেকে ডাটা লোড করব এইজন্য `flow_from_directory` করেছি । পাণ্ডা ডাটা ফ্রেম থেকে লোড করতে হলে `flow_from_dataframe` ব্যাবহার করতে হবে । আপনি `flow_from_directory` এর ফুল ফাংশন ডেফিনেসন এই থেকে দেখে নিতে পারেন । উপরে যে প্যারামিটার ব্যাবহার করা হয়েছে তার কোনটা কি কাজ করছে সটা নিচে দেয়া হল । 

- `directory` ৫ টা ফুলের ডিরেক্টরি যেটা সেটা এখানে দিতে হবে । 
- `batch_size` প্রতিবার কয়টা করে ছবি প্রসেস করে রিটার্ন করবে সেটা বলে দিচ্ছি । 
- `color_mode` ছবি সাদাকালো না রঙ্গিন সেটা বলে দিচ্ছি । 
- `shuffle` ডাটা কি প্রতিবার সাফল হবে কি না সেটা বলে দিচ্ছি । 
- `target_size` আমরা আগে দেখেছিলাম এক একটা ছবির সাইজ এক এক রকম । সেটা ফিক্স একটা সাইজে নিয়ে আসার জন্য টার্গেট সাইজ বলে দিচ্ছি । 
- `classes` ক্লাসের নাম লিস্ট আকারে দিতে হবে । 
- `class_mode='cagegorical'` যেহেতু আমাদের লেবেল ওয়ান হট চাই এই জন্য cagegorical ব্যাবহার করেছি + আমাদের ডাটা ক্যাটাগরিকেল ডাটা। 

## One-Hot এনকোডিং কি 

মনে করা আপনার ৩ টা ক্লাস আছে । ক্লাস গুলো হল A, B, C এই তিনটার লেবেল যথাক্রমে 0, 1, 2 । এখন A এর লেবেল 0 কে আমরা যদি one-hot এনকোডিং এ রিপ্রেজেন্ট করতে চাই তাহলে লিখতে হবে । `label of A = [1, 0, 0]`, `label of B = [0, 1, 0]`, `label of C = [0, 0, 1]` 

এখানে লক্ষণীয়ে যে আমরা যে লিস্ট লেখেছি তার সাইজ ৩ । কারণ আমার মোট ক্লাস ৩ আমাদের যদি ৫ টা ক্লাস থাকে তাহলে সাইজ হবে ৫ অর্থাৎ ক্লাস n হলে সাইজ n হবে । আর একটা বিষয় লক্ষণীয় যে আমাদের লেবেলের মান যত আমরা লিস্টের তত তম ইনডেক্সের মান ১ করে দিয়েছি এবং বাকি ইনডেক্সগুলো মান ০ সেট করে দিয়েছি । 

যখন কোন লেবেল কে এই ভাবে রিপ্রেজেন্ট করা হয় তখন তাকে বলা হয় One-Hot এনকোডিং । এখন প্রশ্ন হল এটা কেন করা হয় ? এটা করা হয় কারণ মডেল ট্রেনিং করার সময় যখন `crossentropy` লস হিসাবে করা হয় তখন লেবেল যদি One-Hot এনকোডিং ফরমেটে থাকে তাহলে লস কম্পিউটেশন খুব দ্রুত হয় । কেন দ্রুত হয় সেটা বুঝতে হলে আপনাকে `crossentropy` কিভাবে কাজ করে সেটা বুঝতে হবে । এই জন্য আপনি <a href="https://www.youtube.com/watch?v=tRsSi_sqXjI"> Cross Entropy</a> এই  ভিডিও দেখতে পারেন । 

In [0]:
# take batch image using `next`
image_batch, label_batch = next(train_data_gen)

for image, label in zip(image_batch[:3], label_batch[:3]):
    print('image shape', image.shape, ' one hot label', label)
    plt.imshow(image)
    plt.show()

এখন আমরা দেখতে পাচ্ছি যে ছবিগুলো আমাদের দেয় সাইজে কনভার্ট হয়ে গিয়েছে এবং এবং লেবেল টা one-hot এ কনভার্ট হয়ে গিয়েছে । এখন আমরা দেখব এটা ব্যাচ (৩২) যতগুলো ছবি আছে সবগুলো প্লট করে দেখি । 

In [0]:
def show_batch(image_batch, label_batch):
    plt.figure(figsize=(16, 8))
    for n in range(32):
        ax = plt.subplot(4, 8, n+1)
        plt.imshow(image_batch[n])
        class_name = CLASS_NAMES[label_batch[n]==1][0].title()
        plt.title(class_name)
        plt.axis('off')

In [0]:
image_batch, label_batch = next(train_data_gen)
show_batch(image_batch, label_batch)

এখন ডাটাটা CNN নেটওয়ার্ক ট্রেনিং করার জন্য ফুল রেডি । উপরের কোড টা প্রতিবার রান করলে দেখবেন ভিন্ন ভিন্ন ছবি আসতেছে । এর কারণ আমরা `shuffle=True` সেট করে দিয়েছি । যার কারণে প্রতিবার ডাটা লোড করার আগে ডাটা সাফল করে নিচ্ছে । এখন আমরা দেখব `tf.data` ব্যাবহার করে কিভাবে ইমেজ ডাটা লোড করা যায় । 

## `tf.data` ব্যাবহার করে ইমেজ ডাটা লোড করা

ইমেজ ডাটা লোড করার ক্ষেত্রে `keras.preprocessing` ব্যাবহার করা সুবিধাজনক কিন্তু এটার কিছু সমস্যা আছে ।  

- এটা `tf.data` থেকে সময় বেশি নেই । সময় কিরকম বেশি নেয় সটা আমরা নিচে দেখিয়েছি । 
- এটাতে প্রোগ্রামারের কন্ট্রোল তুলনামূলক ভাবে কম । 
- এটা টেন্সরফ্লোর সাথে পুরোপুরি ভাবে ইন্টিগ্রেটেড না । 

To load the files as a `tf.data.Dataset` first create a dataset of the file paths:

In [0]:
list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*'))

In [0]:
for f in list_ds.take(5):
    print(type(f.numpy()), ' ', f.numpy())

এখনে আমরা কিছু ফাংশন লিখব । `get_label` এই ফাংশনের কাজ এটাকে কোন ইমেজের ফুল পাথ দিলে সেখান থেকে ইমেজের লেবেল বের করে ।  

In [0]:
def get_label(file_path):
    # convert the path to a list of path components
    parts = tf.strings.split(file_path, os.path.sep)
    # print(parts)
    # The second to last is the class-directory
    return parts[-2] == CLASS_NAMES

In [0]:
file_path = b'/root/.keras/datasets/flower_photos/daisy/16482676953_5296227d40_n.jpg'
tmp_label = get_label(file_path).numpy()
print(tmp_label)

`decode_img` এই ফাংশন কে `tf.io.read_file` দিয়ে ইমেজ ফাইল রিড করে পাঠায়া দিলে সে এটাকে `jpeg` তে ডিকোড করে । এবং শেষ মুল ইমেজ কে রিসাজ করে ফেলে । 

In [0]:
def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])

In [0]:
image = tf.io.read_file(file_path) # read image as string
image = decode_img(image)
image.shape

`process_path` ফাংশন একটা ইমেজ পাথ কে প্রসেস করে `(img, label)` আকারে রিটার্ন করে ।  

In [0]:
def process_path(file_path):
    label = get_label(file_path)
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

`image, label` এই আকারে ডাটা বানানোর জন্য আমরা `Dataset.map` ব্যাবহার করব । একাধিক ইমেজ যাতে করে প্যারালালি লোড এবং প্রসেস করা যায় যায় এই জন্য আমরা `num_parallel_calls=AUTOTUNE)` ব্যাবহার করব । 

`tf.data.experimental.AUTOTUNE` এটা ডিফাইন করে আপনার কম্পিউটারে কাজ করার জন্য কয়টা প্রসেস ফাকা আছে । 

In [0]:
labeled_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

In [0]:
for image, label in labeled_ds.take(1):
    print("Image shape:", image.numpy().shape)
    print("Image label:", label.numpy())
    plt.imshow(image.numpy())
    plt.show()

## ট্রেনিং করানোর মৌলিক কিছু বিষয় 

কোন মডেল এই ডাটা দিয়ে ট্রেনিং করার জন্য নিচের বিষয়গুলো করতে হবে । 

* ডাটা খুব ভালভাবে সাফল করতে হবে 
* টোটাল ডাটা থেকে ব্যাচ অনুসারে ডাটা নেয়া 
* এবং এই ব্যাচ ডাটা যত তাড়াতাড়ি সম্ভব ট্রেনিং জন্য প্রস্তুত করা 

উপরের ৩ টা ফিচার `tf.data` দিয়ে খুব সহজেই ইমপ্লিমেন্ট করা যায় । 

In [0]:
def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
    # ছোট ডাটাসেটের জন্য `ds = ds.cache(cache)` দরকার নাই । কিন্তু ডাটাসেট যদি 
    # অনেক বড় হয় তাহলে RAM এ সব ডাটা একসাথে ধরবে না । তখন আমাদের কে HDD 
    # এর সাহায্য নিতে হবে । 
    if cache:
        # যদি ডাটা অনেক বড় হয় তাহলে তাহলে আমরা ডাটাকে মেমরিতে না রেখে 
        # আমরা যে cache=যে পাথ দিব সেখানে ডাটা ক্যাশ করে রাখবে । 
        # নিচে এটারও আমরা একটা উদাহরণ দেখব । 
        if isinstance(cache, str):
            # HDD এ ডাটা ক্যাশ করে রাখবে । 
            ds = ds.cache(cache)
        else:
            # RAM এ ডাটা ক্যাশ করে রাখবে 
            ds = ds.cache()

    # ডাটা আমরা সাফল করে নিচ্ছি 
    ds = ds.shuffle(buffer_size=shuffle_buffer_size)
    
    # ডাটাসেট অসংখ্যবার রিপিট হবে সেটা বলে দিলাম 
    ds = ds.repeat()

    # BATCH_SIZE অনুসারে আমারা ডাটা নিয়ে নিচ্ছি 
    ds = ds.batch(BATCH_SIZE)

    # যখন মডেল একটা ব্যাচ ডাটার উপর ট্রেনিং হতে থাকবে তখন পরবর্তী ব্যাচ আগে থেকেই প্রস্তুত করে রাখব 
    # যাতে করে মডেলের যখন নতুন ব্যাচ ডাটা দরকার তখন এসে অপেক্ষে করা না লাগে । 
    ds = ds.prefetch(buffer_size=AUTOTUNE)

    return ds

উপরের ফাংশন `ds.shuffle(buffer_size=shuffle_buffer_size)` তে `buffer_size` আসলে কি করতেছে । এটা যেটা করে সেটা হল, মনে করেন আপনার সব মিলে ১০ টা ডাটা আছে । এবং আপনি যদি `buffer_size=3` দেন তাহলে সেটা ১০ টা ডাটা থেকে ৩ টা ডাটা নিবে এবং আপনার `BATCH_SIZE` যদি ২ হয় তাহলে তাহলে সেই ৩ টা ডাটা থেকে ২ টা ডাটা নিবে । এখন আপনার বাফারে কিন্তু ১ টা ডাটা আছে । কারণ ২ টা ডাটা আপনি নিয়ে নিয়েছেন । বাফারে এই দুইটা ফাকা জায়গা পুরন হবে মুল ১০ টা ডটা থেকে নতুন দুইটা ডাটা নিয়ে । এটা সম্পর্কে আরও জানতে দেখেন <a href="https://www.tensorflow.org/api_docs/python/tf/data/Dataset#shuffle">shuffle</a>

`ds.repeat()` এখনে ফাংশন কোন ভেলু দেয়া না হলে সেটা ইনফিনিট ভাবে ডাটা কে রিপিট করবে । কিন্তু ভেলু দিয়ে দিয়ে ভেলু অনুসারে রিপিট করবে । মনে করে আপনার ডাটাসেট এ ৩ টা ডাটা আছে । এই রকম `[1, 2, 3]` এখন আপনি যদি `ds.repeat(2)` দেন তাহলে ডাটা এই রকম হবে `[1, 2, 3, 1, 2, 3]` এটা কাজে লাগে যখন ডাটা শেষ হয়ে যায় কিন্তু আমরা ট্রেনিং প্রসেস চালাতে চাই । রিপিট ফাংশন সম্পর্কে আরও জানতে <a href="https://www.tensorflow.org/api_docs/python/tf/data/Dataset#repeat"> repeat </a>

আমাদের ফাংশন রেডি । এখন আমরা এটাকে ব্যাবহার করতে পারব । নিচের মত করে । 

In [0]:
train_ds = prepare_for_training(labeled_ds)

image_batch, label_batch = next(iter(train_ds))

In [0]:
show_batch(image_batch.numpy(), label_batch.numpy())

## সময়  

এখানে আমরা দেখব কোনটাতে কেমন সময় লাগে । প্রথমে আমরা একটা ফাংশন লিখি যেটা আমাদের কে এই ডাটাসেটের উপর কোনটা কোনটা কেমন সময় লাগতেছে সেটা হিসাব করে দিবে । 

In [0]:
import time
default_timeit_steps = 1000

def timeit(ds, steps=default_timeit_steps):
    start = time.time()
    it = iter(ds)
    for i in range(steps):
        batch = next(it)
        if i%10 == 0:
            print('.',end='')
    print()
    end = time.time()

    duration = end-start
    print("{} batches: {} s".format(steps, duration))
    print("{:0.5f} Images/s".format(BATCH_SIZE*steps/duration))

এখন আমরা উপরের দুইটা ডাটা জেনারেটরে মধ্যে সময় হিসাব করি । 

In [0]:
# `keras.preprocessing`
timeit(train_data_gen)

In [0]:
# `tf.data`
timeit(train_ds)

এখানে আমরা দেখতে পাচ্ছি দুইটার মধ্যে কত পার্থক্য । কিন্তু আমাদের মেমরি যদি কম হয় তাহলে এটা আমরা করতে পারব না । 

একবার প্রসেস করা ডাটা আমরা `.cache` এর মাধ্যমে জমা করে রাখব । পরের বার যখন একই ডাটার জন্য রিকুয়েস্ট করবে তখন আগে দেখবে ডাটা টা ক্যাশ এ আছে কি না । যদি ক্যাশ এ থাকে তাহলে সেটা রিটার্ন করবে । যদি ক্যাশ এ না থাকে তাহলে ডাটা কে প্রসেস করে ক্যাশ জমা রাখবে । যাতে পরবর্তী সময়ে সেটা ব্যাবহার করা যায় । 

ক্যাশ ব্যাবহার না করে দেখি কেমন সময় লাগে । 

In [0]:
uncached_ds = prepare_for_training(labeled_ds, cache=False)
timeit(uncached_ds)

যদি ডাটাসেট মেমরি তে ফিট না করে তাহলে আমরা ক্যাশ ডাটা কে একটা ফাইলে রাইট করে রাখব । এটা RAM এর মত ফাস্ট না হলেও `keras.preprocessing` থেকে অনেক ফাস্ট হবে । 

In [0]:
# create a caches directory
!mkdir caches

filecache_ds = prepare_for_training(labeled_ds, cache="caches/flowers.tfcache")
timeit(filecache_ds)

আমরা `ls` করলে `flowers.tfcache` ফাইল দেখতে পারব । আমাদের ডাটাসেট যত বড় হবে এই ফাইলের সাইজ তত বড় হতে থাকবে । 

In [0]:
!du -sh caches/
print('*'*40)
!ls -l caches/


- মুল লেখা <a href="https://www.tensorflow.org/tutorials/load_data/images"> Image data loading </a>
- <a href="https://youtu.be/kVEOCfBy9uY?list=PLQY2H8rRoyvzIuB8rZXs7pfyjiSUs8Vza"> TF inut pipeline  </a>
- <a href="https://medium.com/analytics-vidhya/tensorflow-2-0-tf-data-api-eaa9889186cc"> TensorFlow 2.0: tf.data API</a>